In [1]:
import numpy as np
import pandas as pd 
from pycaret.regression import *

In [2]:
# importar modelo
modelo = load_model('clientes_gbr_088')

Transformation Pipeline and Model Successfully Loaded


In [3]:
modelo

Pipeline(memory=Memory(location=None),
         steps=[('numerical_imputer',
                 TransformerWrapper(include=['edad', 'imc', 'hijos'],
                                    transformer=SimpleImputer())),
                ('categorical_imputer',
                 TransformerWrapper(include=['genero', 'fumador', 'region'],
                                    transformer=SimpleImputer(strategy='most_frequent'))),
                ('ordinal_encoding',
                 TransformerWrapper(include=['genero', 'fumador'],
                                    tr...
                                                                         'data_type': dtype('O'),
                                                                         'mapping': femenino     0
masculino    1
NaN         -1
dtype: int64},
                                                                        {'col': 'fumador',
                                                                         'data_type': dtype('O'),
                                                                         'mapping': no     0
si     1
NaN   -1
dtype: int64}]))),
                ('onehot_encoding',
                 TransformerWrapper(include=['region'],
                                    transformer=OneHotEncoder(cols=['region'],
                                                              handle_missing='return_nan',
                                                              use_cat_names=True))),
                ('actual_estimator',
                 GradientBoostingRegressor(random_state=100))])

### Paso 1: crear funcion de prediccion

In [4]:
# UDF: User Defined Function / Funcion Definida por el Usuario

def predict(edad, genero, imc, hijos, fumador, region):
    df = pd.DataFrame({
        'edad' : [edad], 
        'genero' : [genero],
        'imc' : [imc], 
        'hijos' : [hijos], 
        'fumador' : [fumador], 
        'region' : [region]
    })
    pred = predict_model(modelo, data=df)
    return {'gastos' : pred['prediction_label'][0].astype('float64')}

In [6]:
predict(48, 'masculino', 32.6, 2, 'no', 'sur')

{'gastos': 10252.358741632775}

### 1.1 Prueba manual de prediccion

In [15]:
ed = input('Edad del paciente: ')

Edad del paciente: 48


In [17]:
im = input('Indice de masa corporal: ')

Indice de masa corporal: 31.54


In [9]:
ge = input('Genero del paciente: ')

Genero del paciente: masculino


In [10]:
hi = input('Cuantos hijos tiene el paciente: ')

Cuantos hijos tiene el paciente: 1


In [11]:
fu = input('El paciente es fumador?: ')

El paciente es fumador?: no


In [12]:
re = input('Region donde vive el paciente: ')

Region donde vive el paciente: oeste


In [18]:
predict(edad=ed, imc=im, genero=ge, hijos=hi, fumador=fu, region=re)

{'gastos': 10355.266228617675}

### Paso 2: crear interfaz Gradio

In [19]:
import gradio as gr

In [20]:
model = modelo

edad = gr.Slider(minimum=1, maximum=100, value=25, label='Edad del paciente')
genero = gr.Dropdown(['masculino', 'femenino'], value='femenino', label='Genero del paciente')
imc = gr.Slider(minimum=5, maximum=40, value=20, label='Indice de Masa Corporal')
hijos = gr.Dropdown([0,1,2,3,4,5,6,7,8,9,10], value=0, label='Numero de hijos')
fumador = gr.Radio(['si', 'no'], value='no', label='El paciente es fumador?')
region = gr.Dropdown(['sur', 'oeste', 'norte', 'este'], value='sur', label='Region de residencia')

In [21]:
gr.Interface(predict, [edad, genero, imc, hijos, fumador, region], 'label', live=True).launch(share=True)

Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.
